## TRY3-           ( FILTER NUMBERS STARTING FROM 64)

# Batch Size- 64
# Image Size- 128X128
# 20 epochs
# best loss: 0.0031

# (*Commendable results either due to the increased image size or the increased filter size)
            

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Input, Dense, Reshape, Conv2DTranspose,\
    Activation, BatchNormalization, ReLU, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import cv2
import time
from tensorflow.keras.utils import Sequence
import datetime
%matplotlib inline

In [ ]:
devices=tf.config.experimental.list_physical_devices('GPU')
print("GPUS: ", len(devices))
tf.config.experimental.set_memory_growth(devices[0],True)

In [ ]:
class DataGenerator(Sequence):

  def __init__(self, base_dir,base_dir2, output_size, shuffle=False, batch_size=10):
    self.base_dir = base_dir
    self.base_dir2 = base_dir2
    self.output_size = output_size
    self.shuffle = shuffle
    self.batch_size = batch_size
    self.all_x = os.listdir(base_dir)
    self.all_y = os.listdir(base_dir2)
    self.on_epoch_end()

  def on_epoch_end(self):
    self.indices = np.arange(len(self.all_x))
    if self.shuffle:
      np.random.shuffle(self.indices)

  def __len__(self):
    return int(len(self.all_x) / self.batch_size)

  def __getitem__(self, idx):
    X = np.empty((self.batch_size, *self.output_size, 3))
    Y = np.empty((self.batch_size, *self.output_size, 3))

    indices = self.indices[idx*(self.batch_size): (idx+1)*(self.batch_size)]
    

    for i,j in enumerate(indices):
      img_path = os.path.join(self.base_dir,self.all_x[j])
      img_path2 = os.path.join(self.base_dir2,self.all_y[j])
        
      img  = cv2.imread(img_path)
      img= cv2.resize(cv2.cvtColor(img,cv2.COLOR_BGR2RGB),self.output_size)
      img2 = cv2.imread(img_path2)
      img2= cv2.resize(cv2.cvtColor(img2,cv2.COLOR_BGR2RGB),self.output_size)
#       print(img_path,img_path2)

      X[i,] = img
      Y[i,] = img2
    X= X.astype('float32')/255
    Y= Y.astype('float32')/255

    return X, Y

In [ ]:
def plotImages(images_arr):
  fig,axes = plt.subplots(1,10,figsize=(20,20))
  axes=axes.flatten()
  for img,ax in zip(images_arr,axes):
    img=img*255
    img=img.astype(np.uint)
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
train = DataGenerator('..\data/train_x/train_x','..\data/train_y/train_y',(128,128), batch_size=64, shuffle=False)
test  = DataGenerator('..\data/test_x/test_x','..\data/test_y/test_y',(128,128), batch_size=64, shuffle=False)

In [ ]:
xx,yy=train[4]
plotImages(xx)
plotImages(yy)

In [ ]:
def conv_operation(x, filters, kernel_size, strides=2):
   x = Conv2D(filters=filters,
              kernel_size=kernel_size,
              strides=strides,
              padding='same')(x)
   x = BatchNormalization()(x)
   x = ReLU()(x)
   return x

def conv_transpose_operation(x, filters, kernel_size):
   x = Conv2DTranspose(filters=filters,
                       kernel_size=kernel_size,
                       strides=2,
                       padding='same')(x)
   x = BatchNormalization()(x)
   x = ReLU()(x)
   return x

def deblurring_autoencoder():
   dae_inputs = Input(shape=(128,128,3), name='dae_input')
   conv_block1 = conv_operation(dae_inputs, 64, 3)
   conv_block2 = conv_operation(conv_block1, 128, 3)
   conv_block3 = conv_operation(conv_block2, 256, 3)
   conv_block4 = conv_operation(conv_block3, 512, 3)
   
   conv_block5 = conv_operation(conv_block4, 512, 3, 1)

   deconv_block1 = conv_transpose_operation(conv_block5, 512,3)
   merge1 = Concatenate()([conv_block3,deconv_block1])
   deconv_block2 = conv_transpose_operation(merge1, 256, 3)
   merge2 = Concatenate()([deconv_block2, conv_block2])
   deconv_block3 = conv_transpose_operation(merge2, 128, 3)
   merge3 = Concatenate()([deconv_block3, conv_block1])
   deconv_block4 = conv_transpose_operation(merge3, 64, 3)

   final_deconv = Conv2DTranspose(filters=3, kernel_size=3,padding='same')(deconv_block4)

#    dae_outputs = Activation('sigmoid', name='dae_output')(final_deconv)
   
   return Model(dae_inputs, final_deconv, name='dae')

In [ ]:
model= deblurring_autoencoder()

In [ ]:
model.summary()

In [ ]:
opt=Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss="mse", metrics=["mae"])

In [5]:
callbacks=[
    ModelCheckpoint('deblur.h5',verbose=1,save_best_only=True,save_weights_only=True),   
]


In [6]:
model.fit(
    train,
    epochs=20,
    verbose=1,
    callbacks=callbacks
)

NameError: name 'model' is not defined

In [ ]:
keras.models.save_model(
    model,'best_model'
)

In [ ]:
xx,yy= test[4]  #randomly chosen
plotImages(xx)
plotImages(yy)

In [ ]:
xx,yy=train[6]

plotImages(xx)
plotImages(yy)

In [ ]:
model=keras.models.load_model('best_model/')

In [ ]:
prediction= model.predict(xx)

In [ ]:
plotImages(prediction)
plotImages(xx)
plotImages(yy)